# Imports

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay
import optuna as opt

# Feature Description

The table results from a query which joins two tables (actuaclly views): "PhotoObj" which contains photometric data and "SpecObj" which contains spectral data.

To ease your start with the data you can read the feature descriptions below:

View "PhotoObj"
objid = Object Identifier
ra = J2000 Right Ascension (r-band)
dec = J2000 Declination (r-band)
Right ascension (abbreviated RA) is the angular distance measured eastward along the celestial equator from the Sun at the March equinox to the hour circle of the point above the earth in question. When paired with declination (abbreviated dec), these astronomical coordinates specify the direction of a point on the celestial sphere (traditionally called in English the skies or the sky) in the equatorial coordinate system.

Source: https://en.wikipedia.org/wiki/Right_ascension

* u = better of DeV/Exp magnitude fit
* g = better of DeV/Exp magnitude fit
* r = better of DeV/Exp magnitude fit
* i = better of DeV/Exp magnitude fit
* z = better of DeV/Exp magnitude fit

The Thuan-Gunn astronomic magnitude system. u, g, r, i, z represent the response of the 5 bands of the telescope.

Further education: https://www.astro.umd.edu/~ssm/ASTR620/mags.html

run = Run Number
rereun = Rerun Number
camcol = Camera column
field = Field number
Run, rerun, camcol and field are features which describe a field within an image taken by the SDSS. A field is basically a part of the entire image corresponding to 2048 by 1489 pixels. A field can be identified by:

run number, which identifies the specific scan,
the camera column, or "camcol," a number from 1 to 6, identifying the scanline within the run, and
the field number. The field number typically starts at 11 (after an initial rampup time), and can be as large as 800 for particularly long runs.
An additional number, rerun, specifies how the image was processed.
View "SpecObj"
specobjid = Object Identifier
class = object class (galaxy, star or quasar object)
The class identifies an object to be either a galaxy, star or quasar. This will be the response variable which we will be trying to predict.

redshift = Final Redshift
plate = plate number
mjd = MJD of observation
fiberid = fiber ID
In physics, redshift happens when light or other electromagnetic radiation from an object is increased in wavelength, or shifted to the red end of the spectrum.

Each spectroscopic exposure employs a large, thin, circular metal plate that positions optical fibers via holes drilled at the locations of the images in the telescope focal plane. These fibers then feed into the spectrographs. Each plate has a unique serial number, which is called plate in views such as SpecObj in the CAS.

Modified Julian Date, used to indicate the date that a given piece of SDSS data (image or spectrum) was taken.

The SDSS spectrograph uses optical fibers to direct the light at the focal plane from individual objects to the slithead. Each object is assigned a corresponding fiberID.

Further information on SDSS images and their attributes:

http://www.sdss3.org/dr9/imaging/imaging_basics.php

http://www.sdss3.org/dr8/glossary.php



# Importing the data

In [3]:
# Import the data in ./../data/

# Data Exploration

In [4]:
# Use df.head and df.info to get a feel for the data

# Categorically Encoding "class"

In [5]:
# Use pandas cat.codes to convert the categorical data to numerical data, this is because xgboost only accepts numerical data as 
# the input in classification problems.

In [6]:
# Define X and y

# Train_test_split 

In [7]:
# Split the data into train and test sets with test_size=0.2 and random_state=42

# Optuna hyperparameter optimalization with cross-validation

In [8]:
def objective(trial):
    param = {
            "n_estimators" : trial.suggest_int('n_estimators', 0, 1000),
            'max_depth':trial.suggest_int('max_depth', 2, 25),
            'reg_alpha':trial.suggest_int('reg_alpha', 0, 5),
            'reg_lambda':trial.suggest_int('reg_lambda', 0, 5),
            'min_child_weight':trial.suggest_int('min_child_weight', 0, 5),
            'gamma':trial.suggest_int('gamma', 0, 5),
            'learning_rate':trial.suggest_loguniform('learning_rate',0.005,0.5),
            'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.1,1,0.01),
            'nthread' : -1
        }

    model = xgb.?(**param)
    score = cross_val_score(?, ?, ?, cv=5, scoring='f1_macro')
    return score.?



SyntaxError: invalid syntax (3353104230.py, line 14)

In [ ]:
# Create a Optuna study object with .create_study() and set the direction to maximize, this is because we want to maximize the f1 score.
# Then, run the study with .optimize() and pass the objective function and the number of trials as parameters.
study = 
study.

: 

# Using optuna hyperparams to make a model

In [ ]:
# Retrieve the best parameters from the study with .best_params
# And create a XGBoost classifier with the best parameters, then fit the model with the training data

: 

# Using cross-validation to get an accurate score

In [ ]:
# Use cross_val_score to get the cross validation score of the model with the training data.
# Cross validation is a technique to evaluate the performance of a model on unseen data, it is done by splitting the data into "k" folds
# and then training the model on "k-1" folds and testing it on the remaining fold. This is repeated "k" times and the average of the
# scores is taken as the final score. This effectively means that the data is trained and validated on all of the data.
# Run cross_val_score with 5 folds and scoring='f1_macro'

scores = cross_val_score(?, ?, ?, cv=?, scoring=?)

# Print the mean and standard deviation of the scores, this is a good way to evaluate the performance of the model.

: 

# Predicting on the test set with the model

In [ ]:
# Predict the test data

: 

# Plot the feature importances

In [ ]:
# XGBoost has a built in function to plot the feature importance, use it to plot the feature importance of the model.
# Feature importance shows how much each feature contributes to the model, this is a good way to see which features are important

xgb.?

: 

# Plotting the confusion matrix

In [ ]:
# A confusion matrix is a table that is often used to describe the performance of a classification model (or "classifier") 
# on a set of test data for which the true values are known. The confusion matrix itself is relatively simple to understand,
# but the related terminology can be confusing. The confusion matrix shows the ways in which your classification model is confused
# when it makes predictions. It gives you insight not only into the errors being made by your classifier but more importantly
# the types of errors that are being made.

# Get the confusion matrix from sklearn.metric.confusion_metric and plot it with ConfusionMatrixDisplay
# Make sure to include display_labels with the class names (HINT: model.classes_) to make the plot more readable



cm = ?(?, ?)
disp = ?(?=?, ?=?)

disp.?(cmap='Blues')

: 

# Plot a tree from the model

In [ ]:
# XGBoost is a tree based ensemble model, it is always a good idea to visualize the tree to see how the model is making the predictions.
# Use xgb.to_graphviz to visualize the tree and save it as a png file.

: 